In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

### a) Basic prompt

In [2]:
from langchain_openai import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [6]:
text = """
Teoria dos jogos ou Teoria de jogos é um ramo da matemática aplicada que estuda situações 
estratégicas onde jogadores escolhem diferentes ações na tentativa de melhorar seu retorno. 
Inicialmente desenvolvida como ferramenta para compreender comportamento econômico e depois 
usada pela Corporação RAND para definir estratégias nucleares, a teoria dos jogos é hoje usada 
em diversos campos acadêmicos. A partir de 1970 a teoria dos jogos passou a ser aplicada ao 
estudo do comportamento animal, incluindo evolução das espécies por seleção natural. Devido a 
interesse em jogos como o dilema do prisioneiro iterado, no qual é mostrada a impotência de 
dois jogadores racionais escolherem algo que beneficie a ambos sem combinado prévio, a teoria 
dos jogos vem sendo aplicada nas ciências políticas, ciências militares, ética, economia, 
filosofia e, recentemente, no jornalismo, área que apresenta inúmeros e diversos jogos, 
tanto competitivos como cooperativos. Finalmente, a teoria dos jogos despertou a atenção 
da ciência da computação que a vem utilizando em avanços na inteligência artificial e cibernética.
"""

messages = [
    SystemMessage(content='You are a expert copywriter with expertie in summarizing documents'),
    HumanMessage(content=f'Please provide a short and concise summary of the follow test: \n TEXT: {text}')
]

llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')

In [5]:
llm.get_num_tokens(text=text)

302

In [7]:
summary_output = llm.invoke(messages)

print(summary_output)

content='Game theory is a branch of applied mathematics that studies strategic situations where players choose different actions to improve their outcomes. Initially developed to understand economic behavior and later used by the RAND Corporation to define nuclear strategies, game theory is now applied in various academic fields. Since the 1970s, it has been used to study animal behavior, including species evolution through natural selection. Game theory is also applied in political science, military science, ethics, economics, philosophy, and journalism. Recently, it has gained attention in computer science for advancements in artificial intelligence and cybernetics.' response_metadata={'token_usage': {'completion_tokens': 113, 'prompt_tokens': 341, 'total_tokens': 454}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-01618dcc-473c-457c-a847-de4344cce8ae-0' usage_metadata={'input_tokens': 341, 'output_tokens': 113, 'to

### Summaring using prompt templates

In [8]:
from langchain import PromptTemplate
from langchain.chains import LLMChain

In [9]:

template = '''
Write a concise and short summary of the following text:
TEXT: `{text}`
Translate the summary to {language}
'''

prompt =  PromptTemplate(
    input_variables=['text', 'language'],
    template=template
)

In [11]:
llm.get_num_tokens(prompt.format(text=text, language='English'))

323

In [12]:
chain = LLMChain(llm=llm, prompt=prompt)

summary = chain.run({'text': text, 'language': 'English'})

print(summary)

Game theory is a branch of applied mathematics that studies strategic situations where players choose different actions in an attempt to improve their return. Initially developed as a tool to understand economic behavior and later used by the RAND Corporation to define nuclear strategies, game theory is now used in various academic fields. Since the 1970s, game theory has been applied to the study of animal behavior, including the evolution of species through natural selection. Due to interest in games like the iterated prisoner's dilemma, where the powerlessness of two rational players to choose something that benefits both without prior agreement is shown, game theory has been applied in political science, military science, ethics, economics, philosophy, and recently in journalism. Finally, game theory has caught the attention of computer science, which has been using it in advances in artificial intelligence and cybernetics.


### Summarizing using StuffDocumentsChain

In [13]:
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document

In [16]:
with open('data/churchill_speech.txt', encoding='utf-8') as file:
    text = file.read()

docs = [Document(page_content=text)]

In [17]:
template = '''
Write a concise and short summary of the following text:
TEXT: `{text}`
'''

prompt =  PromptTemplate(
    input_variables=['text'],
    template=template
) 

In [19]:
chain = load_summarize_chain(
    llm=llm,
    chain_type='stuff',
    prompt=prompt,
    verbose=False
)

output_summary = chain.invoke(docs)

In [21]:
print(output_summary['output_text'])

In his speech "We Shall Fight on the Beaches" delivered on June 4, 1940, Winston Churchill recounts the events leading up to the evacuation of Dunkirk during World War II. He describes the desperate situation faced by the British and French Armies as they were surrounded by German forces, and the heroic efforts made to evacuate troops from the beaches. Churchill emphasizes the importance of air power in the successful evacuation and expresses confidence in the ability of the British Empire to defend their homeland against tyranny, declaring their determination to fight on no matter the cost.


### Sumarizing large documents using map reduce

In [22]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [24]:
with open('data/churchill_speech.txt', encoding='utf-8') as file:
    text = file.read()

recursive_text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

chunks = recursive_text_splitter.create_documents([text])

In [25]:
len(chunks)

23

In [26]:
chain = load_summarize_chain(
    llm=llm,
    chain_type='stuff',
    prompt=prompt,
    verbose=False
)

output_summary = chain.invoke(chunks)

In [27]:
print(output_summary['output_text'])

In his speech "We Shall Fight on the Beaches" delivered on June 4, 1940, Winston Churchill describes the dire situation faced by the British and French Armies during World War II. He recounts the intense battles and sacrifices made to evacuate troops from Dunkirk, highlighting the bravery of the soldiers and the crucial role played by the Royal Air Force and Navy. Despite facing colossal military disasters in France and Belgium, Churchill expresses determination to defend the British Isles against invasion, emphasizing the resolve of the government, Parliament, and the nation to fight on and never surrender.


In [30]:
map_prompt = '''
Write a short and concise summary of the following:
Text: `{text}`
CONciseE SuMMARY
'''

map_prompt_template =  PromptTemplate(
    input_variables=['text'],
    template=map_prompt
) 

In [32]:
combine_prompt = '''
Wriet a consize summary of the fallowing text that covers the key points.
Add a title to the summary.
Start your summary with a INTRODUCTION PARAGRAPH tha gives an overview of the topic FOLLOWED
by BULLET POINTS if possible AND end the summary with a CONCLUSION PHRASE.
Text: `{text}`
'''

combine_prompt_template =  PromptTemplate(
    input_variables=['text'],
    template=combine_prompt
) 

In [34]:
summary_chain = load_summarize_chain(
    llm=llm,
    chain_type='map_reduce',
    map_prompt=map_prompt_template,
    combine_prompt=combine_prompt_template,
    verbose=False
)

output = summary_chain.invoke(chunks)

In [36]:
print(output['output_text'])

Title: Winston Churchill's Speech on the Evacuation of Dunkirk

Introduction:
Winston Churchill's speech on June 4, 1940, highlighted the dire situation faced by the British and French Armies after the French defenses were broken. Despite overwhelming odds, the determination to fight on was emphasized.

Key Points:
- Belgian Army overwhelmed by German forces, leading to the need for a rapid retreat to Dunkirk
- Fierce defense of Boulogne and Calais by British and French forces
- Successful evacuation of over 335,000 troops from Dunkirk by Royal Air Force and Navy
- Importance of recognizing the victory achieved by the Air Force during the evacuation
- Bravery and skill of Royal Air Force pilots in defending against German forces
- Heavy losses suffered in battle, but British Expeditionary Force remains well-equipped
- Need to reconstitute and strengthen the British Expeditionary Force for defensive preparations
- Determination of His Majesty's Government and the nation to defend agains

### Sumarizing CombinedDocumentChain

In [42]:
from langchain.document_loaders import UnstructuredPDFLoader

loader = UnstructuredPDFLoader('data/rag_powered_by_google_search.pdf')


Summarizing using langchain agents

In [46]:
from langchain.agents import initialize_agent, Tool
from langchain.utilities import WikipediaAPIWrapper

In [47]:
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')
wikipedia = WikipediaAPIWrapper()

In [49]:
tools = [
    Tool(
        name='Wikipedia',
        func=wikipedia.run,
        description='Usefull for when you need to get information from wikipedia.'
    )
]

In [50]:
agent_executor = initialize_agent(
    tools=tools,
    llm=llm,
    agent='zero-shot-react-description',
    verbose=True
)

In [51]:
ouptput = agent_executor.invoke('Can you please provide a short summary of George Washington?')



> Entering new AgentExecutor chain...
I should use Wikipedia to get a brief summary of George Washington.
Action: Wikipedia
Action Input: George Washington
Observation: Page: George Washington
Summary: George Washington (February 22, 1732 – December 14, 1799) was an American Founding Father, military officer, and politician who served as the first president of the United States from 1789 to 1797. Appointed by the Second Continental Congress as commander of the Continental Army in 1775, Washington led Patriot forces to victory in the American Revolutionary War and then served as president of the Constitutional Convention in 1787, which drafted the current Constitution of the United States. Washington has thus become commonly known as the "Father of his Country".
Washington's first public office, from 1749 to 1750, was as surveyor of Culpeper County in the Colony of Virginia. In 1752, he received military training and was granted the rank of major in the Virginia Regiment. During the F

In [53]:
print(output['output_text'])

Title: Winston Churchill's Speech on the Evacuation of Dunkirk

Introduction:
Winston Churchill's speech on June 4, 1940, highlighted the dire situation faced by the British and French Armies after the French defenses were broken. Despite overwhelming odds, the determination to fight on was emphasized.

Key Points:
- Belgian Army overwhelmed by German forces, leading to the need for a rapid retreat to Dunkirk
- Fierce defense of Boulogne and Calais by British and French forces
- Successful evacuation of over 335,000 troops from Dunkirk by Royal Air Force and Navy
- Importance of recognizing the victory achieved by the Air Force during the evacuation
- Bravery and skill of Royal Air Force pilots in defending against German forces
- Heavy losses suffered in battle, but British Expeditionary Force remains well-equipped
- Need to reconstitute and strengthen the British Expeditionary Force for defensive preparations
- Determination of His Majesty's Government and the nation to defend agains